In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 21:06:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Question 1: Spark Version
spark.version

'3.5.1'

In [4]:
fhv_schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropOff_datetime", types.TimestampType(), True),
    types.StructField("PUlocationID", types.IntegerType(), True),
    types.StructField("DOlocationID", types.IntegerType(), True),    
    types.StructField("SR_Flag", types.StringType(), True),
    types.StructField("Affiliated_base_number", types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(fhv_schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

df = df.repartition(6)

df.write.parquet('data/pq/fhvhv/2019/10/', mode='overwrite')

In [20]:
# Question 2: Average size for each parquet partition
!ls -lh data/pq/fhvhv/2019/10

total 39M
-rw-r--r-- 1 swong swong    0 Mar  4 21:12 _SUCCESS
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00000-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00001-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00002-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00003-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00004-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet
-rw-r--r-- 1 swong swong 6.4M Mar  4 21:12 part-00005-21811eb7-6be5-4214-8949-807157529877-c000.snappy.parquet


**Q3**: How many taxi trips were there on February 15?

In [26]:
from pyspark.sql import functions as F

In [27]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [28]:
df.registerTempTable('fhvhv_2019_10')

/home/swong/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [52]:
# Question 3: Row Count on October 15th, 2019
spark.sql("""
SELECT
    COUNT(*)
FROM 
    fhvhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [30]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [38]:
# Question 4: Longest trip in October
spark.sql("""
SELECT
    to_date(pickup_datetime) as pickup_date,
    (unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime))/(60*60) as length_hours
FROM fhvhv_2019_10
ORDER BY length_hours DESC LIMIT 3
""").show()

+-----------+-----------------+
|pickup_date|     length_hours|
+-----------+-----------------+
| 2019-10-11|         631152.5|
| 2019-10-28|         631152.5|
| 2019-10-31|87672.44083333333|
+-----------+-----------------+



In [41]:
df_zones = spark.read.parquet('zones')

In [42]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [43]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [44]:
df_zones.registerTempTable('zones')

In [46]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [47]:
df_result = df.join(df_zones, df.PUlocationID == df_zones.LocationID)

In [48]:
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+----------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID|  Borough|            Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+----------------+------------+
|              B02653|2019-10-08 13:53:21|2019-10-08 13:56:24|         264|         247|   NULL|                B02653|       264|  Unknown|              NV|         N/A|
|              B00837|2019-10-12 08:20:51|2019-10-12 08:32:35|         264|         264|   NULL|                B00837|       264|  Unknown|              NV|         N/A|
|              B01315|2019-10-05 15:13:04|2019-10-05 15:19:48|         264|          74|   NULL|                B01315|       264|  Unknown|     

In [50]:
df_result.registerTempTable('fhv_trips_zones')

In [51]:
# Question 6. Least frequent pickup location zone
spark.sql("""        
SELECT
    zone as pickup_zone,
    count(*) as frequency
FROM fhv_trips_zones
GROUP BY 1
ORDER BY 2
""").show()

+--------------------+---------+
|         pickup_zone|frequency|
+--------------------+---------+
|         Jamaica Bay|        1|
|Governor's Island...|        2|
| Green-Wood Cemetery|        5|
|       Broad Channel|        8|
|     Highbridge Park|       14|
|        Battery Park|       15|
|Saint Michaels Ce...|       23|
|Breezy Point/Fort...|       25|
|Marine Park/Floyd...|       26|
|        Astoria Park|       29|
|    Inwood Hill Park|       39|
|       Willets Point|       47|
|Forest Park/Highl...|       53|
|  Brooklyn Navy Yard|       57|
|        Crotona Park|       62|
|        Country Club|       77|
|     Freshkills Park|       89|
|       Prospect Park|       98|
|     Columbia Street|      105|
|  South Williamsburg|      110|
+--------------------+---------+
only showing top 20 rows



In [53]:
spark.stop()